In [1]:
import os
import math
import torch
import wandb
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.model_summary import ModelSummary
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryPrecisionRecallCurve
from torchvision.transforms import v2
from torchinfo import summary
import timm

import datetime

import sklearn
import numpy as np

import optuna

import matplotlib.pyplot as plt

from data.datamodule import BinaryImageDataModule
from training.hyperparameter_tuning import TLOptunaTrainer

### Loading Configuration

In the following steps, we will load the configuration settings using the `load_configuration` function. The configuration is stored in the `config` variable which will be used throughout the script.

In [2]:
from config.load_configuration import load_configuration
config = load_configuration()

PC Name: DESKTOP-LUKAS
Loaded configuration from config/config_lukas.yaml


### Logging in to Weights & Biases (wandb)

Before starting any experiment tracking, ensure you are logged in to your Weights & Biases (wandb) account. This enables automatic logging of metrics, model checkpoints, and experiment configurations. The following code logs you in to wandb:

```python
wandb.login()
```
If you are running this for the first time, you may be prompted to enter your API key.

In [3]:
# Initialize the Wandb logger
wandb.login()

wandb: Currently logged in as: lukas-pelz (HKA-EKG-Signalverarbeitung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Setting Seeds for Reproducibility

To ensure comparable and reproducible results, we set the random seed using the `seed_everything` function from PyTorch Lightning. This helps in achieving consistent behavior across multiple runs of the notebook.

In [4]:
pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility

Seed set to 42


### Checking for GPU Devices

In this step, we check for the availability of GPU devices and print the device currently being used by PyTorch. This ensures that the computations are performed on the most efficient hardware available.

In [5]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Torch Version: ', torch.__version__)
print('Using device: ', device)
if device.type == 'cuda':
    print('Cuda Version: ', torch.version.cuda)
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    torch.set_float32_matmul_precision('high')

Torch Version:  2.7.0+cu128
Using device:  cuda
Cuda Version:  12.8
NVIDIA GeForce RTX 5060 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## Model definitions

Now included in model_transferlearning.py

In [6]:
# timm.list_models('*convnext*')

In [7]:
# def getEfficientNetB4_model(amount_of_trainable_linear_layers=2):
#     """
#     Function to get the EfficientNet B3 model with pretrained weights.
#     Returns:
#         model: A PyTorch model instance of EfficientNet B3.
#     """
#     # Load the EfficientNet B3 model with pretrained weights
#     model = timm.create_model('efficientnet_b4', pretrained=True)
    
#     # Modify the classifier for binary classification
#     num_classes = 1  # For binary classification (OK/NOK)
#     if amount_of_trainable_linear_layers == 1:
#         model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)
#     elif amount_of_trainable_linear_layers == 2:
#         # If two linear layers are trainable, we add an intermediate layer
#         model.classifier = torch.nn.Sequential(
#             torch.nn.Dropout(p=0.2),  # Add dropout for regularization
#             torch.nn.Linear(model.classifier.in_features, 256),  # Intermediate layer
#             torch.nn.ReLU(),  # Activation function
#             torch.nn.Dropout(p=0.2),  # Another dropout layer
#             torch.nn.Linear(256, num_classes)
#         )
    
#     # Freeze all layers except the classifier
#     for param in model.parameters():
#         param.requires_grad = False
#     for param in model.classifier.parameters():
#         param.requires_grad = True
    
#     return model, "TL_EfficientNetB4"

# def getResNet152D_model(amount_of_trainable_linear_layers=2):

#     # Load the EfficientNet B3 model with pretrained weights
#     model = timm.create_model('resnet152d', pretrained=True)
    
#     # Modify the classifier for binary classification
#     num_classes = 1  # For binary classification (OK/NOK)
#     if amount_of_trainable_linear_layers == 1:
#         model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
#     elif amount_of_trainable_linear_layers == 2:
#         # If two linear layers are trainable, we add an intermediate layer
#         model.fc = torch.nn.Sequential(
#             torch.nn.Dropout(p=0.2),                        # Add dropout for regularization
#             torch.nn.Linear(model.fc.in_features, 256),     # Intermediate layer
#             torch.nn.ReLU(),                                # Activation function
#             torch.nn.Dropout(p=0.2),                        # Another dropout layer
#             torch.nn.Linear(256, num_classes)
#         )
    
#     # Freeze all layers except the classifier
#     for param in model.parameters():
#         param.requires_grad = False
#     for param in model.fc.parameters():
#         param.requires_grad = True
    
#     return model, "TL_ResNet50D"

# def getInceptionV4_model(amount_of_trainable_linear_layers=2):

#     # Load the EfficientNet B3 model with pretrained weights
#     model = timm.create_model('inception_v4', pretrained=True)
    
#     # Modify the classifier for binary classification
#     num_classes = 1  # For binary classification (OK/NOK)
#     if amount_of_trainable_linear_layers == 1:
#         model.last_linear = torch.nn.Linear(model.last_linear.in_features, num_classes)
#     elif amount_of_trainable_linear_layers == 2:
#         model.last_linear = torch.nn.Sequential(
#             torch.nn.Dropout(p=0.2),                                # Add dropout for regularization
#             torch.nn.Linear(model.last_linear.in_features, 256),    # Intermediate layer
#             torch.nn.ReLU(),                                        # Activation function
#             torch.nn.Dropout(p=0.2),                                # Another dropout layer
#             torch.nn.Linear(256, num_classes)
#         )
    
#     # Freeze all layers except the classifier
#     for param in model.parameters():
#         param.requires_grad = False
#     for param in model.last_linear.parameters():
#         param.requires_grad = True
    
#     return model, "TL_InceptionV4"

# def getConvNextV2_model(amount_of_trainable_linear_layers=2):

#     # Load the EfficientNet B3 model with pretrained weights
#     model = timm.create_model('convnextv2_base', pretrained=True)
    
#     # Modify the classifier for binary classification
#     num_classes = 1  # For binary classification (OK/NOK)
#     if amount_of_trainable_linear_layers == 1:
#         model.head.fc = torch.nn.Linear(model.head.fc.in_features, num_classes)
#     elif amount_of_trainable_linear_layers == 2:
#         model.head.fc = torch.nn.Sequential(
#             torch.nn.Dropout(p=0.2),                                # Add dropout for regularization
#             torch.nn.Linear(model.head.fc.in_features, 256),    # Intermediate layer
#             torch.nn.ReLU(),                                        # Activation function
#             torch.nn.Dropout(p=0.2),                                # Another dropout layer
#             torch.nn.Linear(256, num_classes)
#         )
    
#     # Freeze all layers except the classifier
#     for param in model.parameters():
#         param.requires_grad = False
#     for param in model.head.fc.parameters():
#         param.requires_grad = True
    
#     return model, "TL_ConvNextV2_base"


In [8]:
# model = timm.create_model('convnextv2_base', pretrained=True)
# print(model)

### Transfer Learning with EfficientNet_B4

In this step, we utilize the EfficientNet_B4 model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the classifier layer to match the number of output classes (`num_classes`). 

We freeze all layers except the classifier to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.

In [6]:
from models.model_transferlearning import TL_EfficientNetB4
import data.custom_transforms as custom_transforms

# model = TransferLearningModule(model)
model = TL_EfficientNetB4(amount_of_trainable_linear_layers=1)
config['image_size'] = 380

# Initialize the Wandb logger
# add time to the name of the experiment
now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

# Initialize wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}",
    config={
        'model': type(model).__name__,
        'dataset': 'DwarfRabbits-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate'],
        'image_size': config['image_size']
    }
)

# Initialize Trainer with wandb logger, using early stopping callback
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/',
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

dm = BinaryImageDataModule(
    data_dir=config['path_to_split_aug_pics'],
    batch_size=config['batch_size'],
    num_workers=6,
    transform=v2.Compose([
            custom_transforms.CenterCropSquare(),
            v2.Resize((config['image_size'], config['image_size'])),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
    persistent_workers=True
)

# Training of the model
trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()

trainer.save_checkpoint('trained_model/' + f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}" + '.ckpt')

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | EfficientNet      | 17.6 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
1.8 K     Trainable params
17.5 M    Non-trainable params
17.6 M    Total params
70.202    Total estimated model params size (MB)
659       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  w

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Data ROC AUC,▁▄▅▆▆▇▇▇▇██████████████
epoch,▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
train_acc,▁▄▅▅▆▆▇▇▇▇▇▇▇▇█████████
train_loss,▆▅▆█▆▇▅▄▆▆▄▅▅▃▁▃▁▂▂▃▂▃▆▁▃▁▄▃▃▁▆▂▁▅▅▂▂▅▆▅
trainer/global_step,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
val_acc,▁▃▄▅▆▆▆▇▇▇██▇███▇█████▇
val_loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▂▁▃▃▅▅▆▄▄▇▆▆▇▅▇▆▅▇▆█▆▆▅
val_recall,▁▄▄▅▅▆▆██▆█▇▆█▇▇█▇▇▆██▇
Validation Data ROC AUC,0.99634
epoch,22


### Transfer Learning with ResNet152D

In this step, we utilize the ResNet152D model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the classifier layer to match the number of output classes (`num_classes`). 

We freeze all layers except the classifier to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.

In [7]:
from models.model_transferlearning import TL_ResNet152D
import data.custom_transforms as custom_transforms

# model = TransferLearningModule(model)
model = TL_ResNet152D(amount_of_trainable_linear_layers=1)
config['image_size'] = 224

# Initialize the Wandb logger
# add time to the name of the experiment
now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

# Initialize wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}",
    config={
        'model': type(model).__name__,
        'dataset': 'DwarfRabbits-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate'],
        'image_size': config['image_size']
    }
)

# Initialize Trainer with wandb logger, using early stopping callback
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/',
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

dm = BinaryImageDataModule(
    data_dir=config['path_to_split_aug_pics'],
    batch_size=config['batch_size'],
    num_workers=6,
    transform=v2.Compose([
            custom_transforms.CenterCropSquare(),
            v2.Resize((config['image_size'], config['image_size'])),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
    persistent_workers=True
)

# Training of the model
trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()

trainer.save_checkpoint('trained_model/' + f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}" + '.ckpt')

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | ResNet            | 58.2 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
2.0 K     Trainable params
58.2 M    Non-trainable params
58.2 M    Total params
232.660   Total estimated model params size (MB)
642       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  w

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Data ROC AUC,▁▄▅▆▆▇▇▇▇▇▇▇▇█▇▇███████▇█▇██▇████
epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇██
train_acc,▁▄▅▆▆▇▇▇▇▇█▇▇█▇█▇████████████████
train_loss,▇█▅▄▅▂▅▂▂▃▂▂▂▂▂▂▂▄▂▃▂▂▁▂▁█▁▄▁▄▂▁▂▂▅▁▃▂▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇██████
val_acc,▁▄▄▅▆▆▆▇▆▇▇▇▆▇▇▇▇▇█▇▇▇▇▇▇▇██▇▇▇▇█
val_loss,█▅▄▄▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
val_precision,▁▂▄▄▇▇▇▆█▅▆▆▇▇█▇▇▇▇▇▆▇▇▇▇▆▇▇▇▇▇█▇
val_recall,▁▅▄▇▄▃▃█▁▇██▄▆▃▆▅▆▇▅▇▆▆▅▇▇▆▇▅▆▆▄▆
Validation Data ROC AUC,0.99861
epoch,32


### Transfer Learning with Inception V4

In this step, we utilize the Inception V4 model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the classifier layer to match the number of output classes (`num_classes`). 

We freeze all layers except the classifier to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.


In [8]:
from models.model_transferlearning import TL_InceptionV4
import data.custom_transforms as custom_transforms

# model = TransferLearningModule(model)
model = TL_InceptionV4(amount_of_trainable_linear_layers=1)
config['image_size'] = 299

# Initialize the Wandb logger
# add time to the name of the experiment
now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

# Initialize wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}",
    config={
        'model': type(model).__name__,
        'dataset': 'DwarfRabbits-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate'],
        'image_size': config['image_size']
    }
)

# Initialize Trainer with wandb logger, using early stopping callback
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/',
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

dm = BinaryImageDataModule(
    data_dir=config['path_to_split_aug_pics'],
    batch_size=config['batch_size'],
    num_workers=6,
    transform=v2.Compose([
            custom_transforms.CenterCropSquare(),
            v2.Resize((config['image_size'], config['image_size'])),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
    persistent_workers=True
)

# Training of the model
trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()

trainer.save_checkpoint('trained_model/' + f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}" + '.ckpt')

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | InceptionV4       | 41.1 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
1.5 K     Trainable params
41.1 M    Non-trainable params
41.1 M    Total params
164.577   Total estimated model params size (MB)
836       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*ar

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Data ROC AUC,▁▄▅▆▇▇▇▇▇██▇████████▇██
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
train_acc,▁▅▆▆▆▇▇▇▇▇▇▇▇██████████
train_loss,█▅▅▄▄▄▃█▆▅▂▂▃▂▅▃▂▆▄▃▂▃▂▃▂▂▃▁▁▃▃▂▂▂▃▃▄▁▁▂
trainer/global_step,▁▁▁▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
val_acc,▁▄▅▆▆▇▇▇▇███▇█▇█▇███▇▇█
val_loss,█▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▁▂▃▄▃▅▆▂▅█▅▅▆▅▆▅▅▆▄▆▄▅▅
val_recall,▁▄▅▅▇▆▅█▆▅▇▇▆▇▆▇▇▆▇▆▇▇▇
Validation Data ROC AUC,0.99529
epoch,22


### Transfer Learning with ConvNextV2

In this step, we utilize the ConvNextV2 model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the classifier layer to match the number of output classes (`num_classes`). 

We freeze all layers except the classifier to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.

In [6]:
from models.model_transferlearning import TL_ConvNextV2
import data.custom_transforms as custom_transforms

# model = TransferLearningModule(model)
model = TL_ConvNextV2(amount_of_trainable_linear_layers=1)
config['image_size'] = 224

# Initialize the Wandb logger
# add time to the name of the experiment
now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

# Initialize wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}",
    config={
        'model': type(model).__name__,
        'dataset': 'DwarfRabbits-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate'],
        'image_size': config['image_size']
    }
)

# Initialize Trainer with wandb logger, using early stopping callback
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/',
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

dm = BinaryImageDataModule(
    data_dir=config['path_to_split_aug_pics'],
    batch_size=config['batch_size'],
    num_workers=6,
    transform=v2.Compose([
            custom_transforms.CenterCropSquare(),
            v2.Resize((config['image_size'], config['image_size'])),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
    persistent_workers=True
)

# Training of the model
trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()

trainer.save_checkpoint('trained_model/' + f"{config['wandb_experiment_name']}_{type(model).__name__}_img{config['image_size']}_{current_time}" + '.ckpt')

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | ConvNeXt          | 87.7 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
1.0 K     Trainable params
87.7 M    Non-trainable params
87.7 M    Total params
350.775   Total estimated model params size (MB)
471       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  w

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Data ROC AUC,█▇▆▇▆▇▁▂▂▁▃
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
train_acc,▁▆▇▇▇▇▇████
train_loss,▆▄▂▂▂█▃▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▅▁▁
trainer/global_step,▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
val_acc,▄▁▂█▁▇▁▁▅▇▇
val_loss,▇▃▃▁▃▁▆█▆▆▅
val_precision,▁▄█▅▆▆▆▆▅▅▅
val_recall,▇▃▁█▁▆▁▁▅▆▇
Validation Data ROC AUC,0.99976
epoch,10


### EfficientNet B4: Training and Logging with Weights & Biases including Hyper-Parameter Tuning

In this step, we initialize the Weights & Biases (wandb) logger to track experiment metrics, hyperparameters, and model checkpoints. The logger is configured with project and experiment names, as well as key training parameters such as dataset, batch size, maximum epochs, and learning rate.

We then set up the PyTorch Lightning `Trainer` with the wandb logger and an early stopping callback to monitor validation loss. The model is trained using the specified datamodule, and all relevant metrics are automatically logged to wandb for further analysis and visualization. After training, wandb logging is finalized to ensure all data is properly saved.

In [ ]:
import datetime
config['sweep_id'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

from models.model_transferlearning import getEfficientNetB4_model

config['image_size'] = 300
def objective(trial):
    model_fct = getEfficientNetB4_model
    trainer = TLOptunaTrainer(
        model=model_fct,                        # Function to create the model
        config=config,
        normalize_mean=[0.485, 0.456, 0.406], 
        normalize_std=[0.229, 0.224, 0.225],
        dataset_name="DwarfRabbits-binary"
    )
    return trainer.run_training(trial)

# Create an Optuna study
study = optuna.create_study(direction="minimize")  # because we minimize val_loss

# Set verbosity to WARNING to reduce output clutter
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Start the hyperparameter optimization
study.optimize(objective, n_trials=config['number_of_trials'])

# Best result
print("Best trial:")
print(study.best_trial.params)
print("Best value (val_loss):", study.best_value)

[I 2025-06-02 14:50:50,658] A new study created in memory with name: no-name-d9e51de7-83c3-44ad-b606-3fae3ca32208
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Using 16bit Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | EfficientNet      | 17.6 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
1.8 K     Trainable params
17.5 M    Non-trainable params
17.6 M    Total params
70.202    Total estimated model params size (MB)
659       Modules in train mode
0         Modules in eval mode
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed

In [30]:
optuna.visualization.plot_optimization_history(study)

NameError: name 'study' is not defined

In [9]:
optuna.visualization.plot_param_importances(study)

### ResNet152D: Training and Logging with Weights & Biases including Hyper-Parameter Tuning

In this step, we initialize the Weights & Biases (wandb) logger to track experiment metrics, hyperparameters, and model checkpoints. The logger is configured with project and experiment names, as well as key training parameters such as dataset, batch size, maximum epochs, and learning rate.

We then set up the PyTorch Lightning `Trainer` with the wandb logger and an early stopping callback to monitor validation loss. The model is trained using the specified datamodule, and all relevant metrics are automatically logged to wandb for further analysis and visualization. After training, wandb logging is finalized to ensure all data is properly saved.

In [6]:
from models.model_transferlearning import getResNet152D_model

import datetime
config['sweep_id'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

config['image_size'] = 224
def objective(trial):
    model = getResNet152D_model
    trainer = TLOptunaTrainer(
        model=model,                        # Function to create the model
        config=config,
        normalize_mean=[0.485, 0.456, 0.406], 
        normalize_std=[0.229, 0.224, 0.225],
        dataset_name="DwarfRabbits-binary"
    )
    return trainer.run_training(trial)

# Create an Optuna study
study = optuna.create_study(direction="minimize")  # because we minimize val_loss

# Set verbosity to WARNING to reduce output clutter
optuna.logging.set_verbosity(optuna.logging.INFO)

# Start the hyperparameter optimization
study.optimize(objective, n_trials=config['number_of_trials'], gc_after_trial=True, show_progress_bar=True)

# Best result
print("Best trial:")
print(study.best_trial.params)
print("Best value (val_loss):", study.best_value)

[I 2025-06-17 08:55:39,206] A new study created in memory with name: no-name-9d4522ad-ba85-4a74-b556-eb25f81f59a7


  0%|          | 0/20 [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | ResNet            | 58.7 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
524 K     Trainable params
58.2 M    Non-trainable params
58.7 M    Total params
234.751   Total estimated model params size (MB)
647       Modules in train mode
0         Modules in eval mode
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless

Validation Data ROC AUC,▁▄▅▆▆▇▇▇▇▇▇████████████████
epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇██
train_acc,▁▆▆▇▇▇▇▇█▇█████████████████
train_loss,█▆▄▅▅▃▃▄▃▃▄▃▃▂▂▂▂▃▃▂▂▂▃▂▂▂▂▂▁▂▃▂▂▃▃▂▂▂▂▁
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
val_acc,▁▃▄▅▆▆▇▇▇▇▇▇█▇▇▇▇█▇████████
val_loss,█▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▁▃▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█▇▇▇▆▇▆▇▇
val_recall,▂▂▁▄▄▄▆▆▅▅▆▅▇▆▅▄▄▅▃▇█▆█▇█▆▇
Validation Data ROC AUC,0.99758
epoch,26


...

Optuna Validation loss: 0.03849393501877785
[I 2025-06-17 15:43:59,922] Trial 18 finished with value: 0.03849393501877785 and parameters: {'batch_size': 64, 'image_size': 256, 'max_epochs': 34, 'accumulate_grad_batches': 2, 'precision': '16-mixed', 'optimizer': 'AdamW', 'learning_rate': 0.00423891290896787, 'weight_decay': 5.9666017665336866e-05, 'scheduler': 'StepLR', 'model_classifier_layers': 2}. Best is trial 12 with value: 0.03660617768764496.


c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Using 16bit Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | ResNet            | 58.7 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
524 K     Trainable params
58.2 M    Non-trainable params
58.7 M    Total params
234.751   Total estimated model params size (MB)
647       Modules in train mode
0         Modules in eval mode
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless

Validation Data ROC AUC,▁▄▃▇▇▇▇▇█
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇████
train_acc,▁▆▆▇▇████
train_loss,█▄▄▅▅▄▄▃▂▃▃▇▁▄▃▃▂▃▃▃▁▆▂▃▂▄▂▁▂▁▁▁▁▁▂▁▁▁▂▇
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇█
val_acc,▁▄▁▆▅▇▆█▇
val_loss,▅▃█▁▄▁▃▁▁
val_precision,▄▇▁▇▄▇█▇▆
val_recall,▂▁█▃█▃▁▅▇
Validation Data ROC AUC,0.99844
epoch,8


Optuna Validation loss: 0.05388827994465828
[I 2025-06-17 15:49:57,531] Trial 19 finished with value: 0.05388827994465828 and parameters: {'batch_size': 64, 'image_size': 192, 'max_epochs': 35, 'accumulate_grad_batches': 2, 'precision': '16-mixed', 'optimizer': 'AdamW', 'learning_rate': 0.004531271602426265, 'weight_decay': 6.593933792068769e-06, 'scheduler': 'StepLR', 'model_classifier_layers': 2}. Best is trial 12 with value: 0.03660617768764496.
Best trial:
{'batch_size': 64, 'image_size': 256, 'max_epochs': 24, 'accumulate_grad_batches': 1, 'precision': 32, 'optimizer': 'AdamW', 'learning_rate': 0.0024603367683237068, 'weight_decay': 0.00016334996012291853, 'scheduler': 'StepLR', 'model_classifier_layers': 2}
Best value (val_loss): 0.03660617768764496


In [7]:
optuna.visualization.plot_optimization_history(study)

In [8]:
optuna.visualization.plot_param_importances(study)

### ConvNextV2: Training and Logging with Weights & Biases including Hyper-Parameter Tuning

In this step, we initialize the Weights & Biases (wandb) logger to track experiment metrics, hyperparameters, and model checkpoints. The logger is configured with project and experiment names, as well as key training parameters such as dataset, batch size, maximum epochs, and learning rate.

We then set up the PyTorch Lightning `Trainer` with the wandb logger and an early stopping callback to monitor validation loss. The model is trained using the specified datamodule, and all relevant metrics are automatically logged to wandb for further analysis and visualization. After training, wandb logging is finalized to ensure all data is properly saved.

In [9]:
from models.model_transferlearning import getConvNextV2_model

import datetime
config['sweep_id'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

config['image_size'] = 224
def objective(trial):
    model_fct = getConvNextV2_model
    trainer = TLOptunaTrainer(
        model=model_fct,                        # Function to create the model
        config=config,
        normalize_mean=[0.485, 0.456, 0.406], 
        normalize_std=[0.229, 0.224, 0.225],
        dataset_name="DwarfRabbits-binary"
    )
    return trainer.run_training(trial)

# Create an Optuna study
study = optuna.create_study(direction="minimize")  # because we minimize val_loss

# Set verbosity to WARNING to reduce output clutter
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Start the hyperparameter optimization
study.optimize(objective, n_trials=config['number_of_trials'])

# Best result
print("Best trial:")
print(study.best_trial.params)
print("Best value (val_loss):", study.best_value)

[I 2025-06-17 16:53:37,710] A new study created in memory with name: no-name-df51e347-fc52-494d-92ff-c210c62a565b
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning:

The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | ConvNeXt          | 88.0 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
262 K     Trainable params
87.7 M    Non-trainable params
88.0 M    Total params
351.822   Total estimated model params size (MB)
476       Modules in train mode
0         Modules in eval mode
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_ranking.py:1179: UndefinedMetricWarning:

No negative samples in y_true, false positive value should be meaningles

Validation Data ROC AUC,█▁▅▅
epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆███████████
train_acc,▁▇▇█
train_loss,▃▁▁▂█▁▁▁▁▁▇▃▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇█████
val_acc,██▁▁
val_loss,▁▅█▇
val_precision,▁▂█▄
val_recall,█▇▁▃
Validation Data ROC AUC,0.99975
epoch,3


Optuna Validation loss: 0.026944497600197792


c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning:

The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Optuna Validation loss: 0.021624010056257248


c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning:

The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.

Using 16bit Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | ConvNeXt          | 87.7 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | sigmoid        | Sigmoid           | 0      | train
3 | train_accuracy | BinaryAccuracy    | 0      | train
4 | val_accuracy   | BinaryAccuracy    | 0      | train
5 | val_precision  | BinaryPrecision   | 0      | train
6 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
1.0 K     Trainable params
87.7 M    Non-trainable params
87.7 M    Total params
350.775   Total estimated model params size (MB)
471       Modules in train mode
0         Modules in eval mode
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning:

A single label was found in 'y_true' and 'y_pred'. For the confusion matrix t

Validation Data ROC AUC,▅██▁▅▆
epoch,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▄▄▄▄▄▅▅▅▅▅▅▇▇▇▇▇▇████████
train_acc,▁▇▇███
train_loss,█▂▅▅▃▂▂▁▂▂▂▁▁▁▂▁▅▁▂▂▁▁▂▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▄
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇█████
val_acc,▁▇█▄▂▃
val_loss,▄▁▁█▄▂
val_precision,▄▄▅█▄▁
val_recall,▁▇█▁▂▆
Validation Data ROC AUC,0.99979
epoch,5


Optuna Validation loss: 0.019135424867272377
Best trial:
{'batch_size': 64, 'image_size': 256, 'max_epochs': 23, 'accumulate_grad_batches': 1, 'precision': '16-mixed', 'optimizer': 'SGD', 'learning_rate': 0.0042486420781319215, 'weight_decay': 0.0014540551521945075, 'scheduler': 'StepLR', 'model_classifier_layers': 1}
Best value (val_loss): 0.017790257930755615


In [12]:
optuna.visualization.plot_optimization_history(study)

In [13]:
optuna.visualization.plot_param_importances(study)